# **Preprocessing the Input Features** 

Se puede preprocesar los datos sobre la marcha al cargarlos con DATA API (por ejemplo, utilizando el método map() del conjunto de datos,como vimos anteriormente), o puede incluir una capa de
preprocesamiento directamente en su modelo. Veamos ahora esta última opción.

## Encoding Categorical Features Using One-Hot Vectors
Consider the **ocean_proximity** feature in the California housing dataset

In [1]:
import pandas as pd

df =pd.read_csv("resources\housing.csv")
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


* Se crean  un inicializador para lookup table(tabla de
búsqueda), pasándole la lista de categorías y sus índices
correspondientes. En este ejemplo, ya tenemos estos datos, por lo
que utilizamos un inicializador `KeyValueTensorInitializer`; pero si las categorías estuvieran
listadas en un fichero de texto (con una categoría por línea),
utilizaríamos en su lugar un `TextFileInitializer`.

* En las dos últimas líneas se crean la **lookup table**, le damos
el inicializador y especificamos el número de **f out-of-vocabulary (oov) buckets**. Si buscamos una categoría que no existe en el
vocabulario, la tabla de búsqueda calculará un hash de esta
categoría y lo utilizará para asignar la categoría desconocida a
uno de los buckets oov. Sus índices comienzan después de las
categorías conocidas, por lo que en este ejemplo los índices de
los dos cubos oov son 5 y 6.

In [3]:
import tensorflow as tf

# define the vocabulary 
vocab = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]

# create a  tensor with corresponding inices(0,4)
indices = tf.range(len(vocab), dtype=tf.int64)

table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)

num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

In [4]:
# un ejemplo de categorias
categories = tf.constant(["NEAR BAY", "DESERT", "INLAND", "INLAND"])
cat_indices = table.lookup(categories)
cat_indices

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([3, 5, 1, 1], dtype=int64)>

In [5]:
cat_one_hot = tf.one_hot(cat_indices, depth=len(vocab) + num_oov_buckets)
cat_one_hot


<tf.Tensor: shape=(4, 7), dtype=float32, numpy=
array([[0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.]], dtype=float32)>

>Notar como la categoria desierto fue mapeada en uno de los dos  **oov buckets** (con el indice 5)
 
Este mismo comportamiento se puede lograr usando la capa `tf.keras.layers.TextVectorization()`

## Encoding Categorical Features Using Embeddings

 El tamaño de cada one-hot vector es la longitud del vocabulario más el número de oov buckets. Esto
está bien cuando sólo hay unas pocas categorías posibles, pero si el vocabulario es grande, es mucho más eficiente codificarlas utilizando
`Embeddings` en su lugar.

Una `Embeddings` es un vector denso entrenable que representa una
categoría. Por defecto, las incrustaciones se inicializan aleatoriamente,
por lo que, por ejemplo, la categoría "NEAR BAY" podría estar
representada inicialmente por un vector aleatorio como [0,131, 0,890],
mientras que la categoría "CERCA DEL OCÉANO" podría estar
representada por otro vector aleatorio como [0,631, 0,791]. En este
ejemplo, utilizamos Embeddings2D, pero el número de dimensiones es
un hiperparámetro que se puede ajustar. Como estas incrustaciones son
entrenables, mejorarán gradualmente durante el entrenamiento el Descenso del Gradeinte colocara
 las categorias similares mas cercas y las 
q difieren mucho, mas alejadas;
 
Cuanto mejor sea la representación, más fácil le resultará a la red
neuronal hacer predicciones precisas, por lo que el entrenamiento tiende
a hacer de las incrustaciones representaciones útiles de las categorías.
Esto se llama **`representation learning`**

In [7]:
embedding_dim = 2
embed_init = tf.random.uniform([len(vocab) + num_oov_buckets, embedding_dim])
embedding_matrix = tf.Variable(embed_init)
embedding_matrix

<tf.Variable 'Variable:0' shape=(7, 2) dtype=float32, numpy=
array([[0.77735114, 0.8121606 ],
       [0.5239444 , 0.54260087],
       [0.4377067 , 0.46367276],
       [0.7861433 , 0.9337244 ],
       [0.9940556 , 0.9592352 ],
       [0.5111915 , 0.8097472 ],
       [0.7584634 , 0.23076165]], dtype=float32)>

In [8]:
categories = tf.constant(["NEAR BAY", "DESERT", "INLAND", "INLAND"])
cat_indices = table.lookup(categories)
cat_indices

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([3, 5, 1, 1], dtype=int64)>

In [9]:
tf.nn.embedding_lookup(embedding_matrix, cat_indices)

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[0.7861433 , 0.9337244 ],
       [0.5111915 , 0.8097472 ],
       [0.5239444 , 0.54260087],
       [0.5239444 , 0.54260087]], dtype=float32)>

Keras ya proporciona una capa `keras.layers.Embedding` que maneja la matriz de incrustación (trainable by default);

In [12]:
embedding = tf.keras.layers.Embedding(input_dim=len(vocab) +
num_oov_buckets, output_dim=embedding_dim)
embedding(cat_indices)

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.02801899,  0.04836044],
       [ 0.01771268, -0.04975939],
       [ 0.03186648,  0.00765382],
       [ 0.03186648,  0.00765382]], dtype=float32)>

Veamos un ejemplo de como usarlo en una red 

In [13]:
from tensorflow import keras
# input of categories
categories = keras.layers.Input(shape=[], dtype=tf.string)
cat_indices = keras.layers.Lambda(lambda cats: table.lookup(cats))(categories)
cat_embed = keras.layers.Embedding(input_dim=6, output_dim=2)(cat_indices)

# input numerical features
regular_inputs = keras.layers.Input(shape=[8])

encoded_inputs = keras.layers.concatenate([regular_inputs, cat_embed])

outputs = keras.layers.Dense(1)(encoded_inputs)

model = keras.models.Model(inputs=[regular_inputs, categories], outputs=[outputs])
